### Goal 1: What movie genres are the most profitable?
- rt_movie_info merged with bom_movie_gross OR movie_basics

### Goal 2: What directors worked on the most profitable films?
- rt_,movie_info merged with bom_movie_gross OR movie_basics

### Goal 3: What is the most profitable release date for a movie?
- tn_mb

In [1]:
# import necessary libraries

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

conn = sqlite3.Connection("im.db")

Add the necessary datasets and plot them on a pandas DataFrame

In [2]:
# For Goal 1-2
bom_movie_gross = pd.read_csv("zippedData/bom.movie_gross.csv.gz")

conn = sqlite3.connect("im.db")
movie_basics = pd.read_sql("SELECT * FROM movie_basics;", conn)

# For Goal 3
tn_mb = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')

Drop records that have no genres associated since it only makes up less than 4% of the total records in the original movie_basics table.


In [3]:
#    p.primary_name
#JOIN persons AS p
#    ON p.person_id = d.movie_id
#JOIN directors AS d
#    
q = """
SELECT
    mb.primary_title,
    mb.genres,
    p.primary_name
FROM movie_basics AS mb
JOIN directors AS d
    ON mb.movie_id = d.movie_id
JOIN persons AS p
    ON d.person_id = p.person_id
GROUP BY mb.primary_title
HAVING primary_profession LIKE '%director%'
"""
q2 = """
SELECT 
    p.primary_name,
    p.primary_profession
FROM persons AS p
WHERE primary_profession LIKE '%director%';
"""
imdb = pd.read_sql(q, conn)
print(imdb.info())
imdb.head()

#imdb2 = pd.read_sql(q2,conn)
#print(imdb.info())
#imdb2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121179 entries, 0 to 121178
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   primary_title  121179 non-null  object
 1   genres         118367 non-null  object
 2   primary_name   121179 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB
None


,primary_title,genres,primary_name
0,!Women Art Revolution,Documentary,Lynn Hershman-Leeson
1,#1 Serial Killer,Horror,Stanley Yung
2,#5,"Biography,Comedy,Fantasy",Ricky Bardy
3,#50Fathers,Comedy,Joddy Eric Matthews
4,#66,Action,Asun Mawardi


## rt_movie_info
### Step 1: Remove unecessary columns
- Since I plan on using this DataFrame to identify the **director** and then use it to join it to other DataFrames I am only keeping the following columns
    - id
    - synopsis
    - rating
    - writer
    - thea

In [23]:
def str_to_list(string):
    li = list(string.split(","))
    return li

str_to_list(imdb['genres'])
imdb['genres'][2]

AttributeError: 'Series' object has no attribute 'split'

In [4]:
imdb['genres'] = imdb['genres'].str.split(',')

<class 'list'>


,primary_title,genres,primary_name
0,!Women Art Revolution,[Documentary],Lynn Hershman-Leeson
1,#1 Serial Killer,[Horror],Stanley Yung
2,#5,"[Biography, Comedy, Fantasy]",Ricky Bardy
3,#50Fathers,[Comedy],Joddy Eric Matthews
4,#66,[Action],Asun Mawardi


In [18]:
print(type(imdb['genres'][1]))
imdb.head()

<class 'list'>


,primary_title,genres,primary_name
0,!Women Art Revolution,[Documentary],Lynn Hershman-Leeson
1,#1 Serial Killer,[Horror],Stanley Yung
2,#5,"[Biography, Comedy, Fantasy]",Ricky Bardy
3,#50Fathers,[Comedy],Joddy Eric Matthews
4,#66,[Action],Asun Mawardi


## Data Cleaning

### bom_movie_gross:
#### Step 1: Identify what the NaN records under bom_movie_gross['foreign_gross'] refer to
- I created a new DataFrame called **bom_fg_na** that copies over the original **bom_movie_gross** DataFrame to make editing the file a little easier. 
- Results show that out of the 3387 records in the **bom_movie_gross** DataFrame 1350 records have a NaN value under the 'foreign_gross' column.
- This indicates that there is a large number of films that were not screened outside their domestic theaters.
- Upon investigating on the value of keeping these records I found that there was a reasonable spread of data under the 'domestic_gross' column to keep it.

### Step 2: Remove unecessary columns
- I created a new table variable called ***new_bmg**
- I removed the following columns
    - studio
    - year
    
#### Step 3: Convert the NaN records under bom_movie_gross['foreign_gross'] to 0 
- I created a new DataFrame called **bmg_fill_na** that copies over the original **bom_movie_gross** DataFrame to make editing the file a little easier.

### Step 4: Convert **bmg_fill_na['foreign_gross']** to an float64 type so that I may apply operations to it.

### Step 5: Create a new column called 'total_gross' that sums 'domestic_gross' and 'foreign_gross'.



In [16]:
rt_movie_info

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [5]:
print(bom_movie_gross.info())
bom_movie_gross.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010


In [6]:
# Code for cleaning the bom_movie_gross DataFrame

# Step 1
bom_fg_na = bom_movie_gross[bom_movie_gross['foreign_gross'].isna()]
print(bom_fg_na.info())
bom_fg_na

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1350 entries, 222 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           1350 non-null   object 
 1   studio          1349 non-null   object 
 2   domestic_gross  1350 non-null   float64
 3   foreign_gross   0 non-null      object 
 4   year            1350 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 63.3+ KB
None


,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [7]:
# Step 2
bom_movie_gross = bom_movie_gross.drop(['studio', 'year'], axis = 1)
bom_movie_gross

,title,domestic_gross,foreign_gross
0,Toy Story 3,415000000.0,652000000
1,Alice in Wonderland (2010),334200000.0,691300000
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000
3,Inception,292600000.0,535700000
4,Shrek Forever After,238700000.0,513900000
...,...,...,...
3382,The Quake,6200.0,NaN
3383,Edward II (2018 re-release),4800.0,NaN
3384,El Pacto,2500.0,NaN
3385,The Swan,2400.0,NaN


In [8]:
# Step 3
bmg_fill_na = bom_movie_gross

bmg_fill_na['foreign_gross'] = bmg_fill_na['foreign_gross'].fillna(0)
print(bmg_fill_na.info())
bmg_fill_na

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   3387 non-null   object 
dtypes: float64(1), object(2)
memory usage: 79.5+ KB
None


,title,domestic_gross,foreign_gross
0,Toy Story 3,415000000.0,652000000
1,Alice in Wonderland (2010),334200000.0,691300000
2,Harry Potter and the Deathly Hallows Part 1,296000000.0,664300000
3,Inception,292600000.0,535700000
4,Shrek Forever After,238700000.0,513900000
...,...,...,...
3382,The Quake,6200.0,0
3383,Edward II (2018 re-release),4800.0,0
3384,El Pacto,2500.0,0
3385,The Swan,2400.0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   float64
 4   year            3387 non-null   int64  
 5   total_gross     2009 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 158.9+ KB
None


In [9]:
# Step 4
bmg_fill_na['foreign_gross'] = bmg_fill_na['foreign_gross'].str.replace(',','')
bmg_fill_na['foreign_gross'] = bmg_fill_na['foreign_gross'].astype(float)
bmg_fill_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   float64
dtypes: float64(2), object(1)
memory usage: 79.5+ KB


,title,studio,domestic_gross,foreign_gross,year
222,Flipped,WB,1800000.0,NaN,2010
254,The Polar Express (IMAX re-issue 2010),WB,673000.0,NaN,2010
267,Tiny Furniture,IFC,392000.0,NaN,2010
269,Grease (Sing-a-Long re-issue),Par.,366000.0,NaN,2010
280,Last Train Home,Zeit.,288000.0,NaN,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [10]:
# Step 5
bmg_fill_na['total_gross'] = (bmg_fill_na['domestic_gross'] + bmg_fill_na['foreign_gross']).round(2)
# Removes trailing 0 
pd.set_option('display.float_format','{:.2f}'.format)
print(bmg_fill_na.info())
bmg_fill_na

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   domestic_gross  3359 non-null   float64
 2   foreign_gross   2037 non-null   float64
 3   total_gross     2009 non-null   float64
dtypes: float64(3), object(1)
memory usage: 106.0+ KB
None


,title,domestic_gross,foreign_gross,total_gross
0,Toy Story 3,415000000.00,652000000.00,1067000000.00
1,Alice in Wonderland (2010),334200000.00,691300000.00,1025500000.00
2,Harry Potter and the Deathly Hallows Part 1,296000000.00,664300000.00,960300000.00
3,Inception,292600000.00,535700000.00,828300000.00
4,Shrek Forever After,238700000.00,513900000.00,752600000.00
...,...,...,...,...
3382,The Quake,6200.00,NaN,NaN
3383,Edward II (2018 re-release),4800.00,NaN,NaN
3384,El Pacto,2500.00,NaN,NaN
3385,The Swan,2400.00,NaN,NaN


### movie_basics:
### Step 1: Drop unecessary columns
- I removed the following columns
    - movie_id
    - original_title
    - start_year
    - runtime_minutes

In [11]:
print(movie_basics.info())
movie_basics.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB
None


,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.00,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.00,"Biography,Drama"


In [12]:
# Step 1:
movie_basics = movie_basics.drop(['movie_id', 'original_title', 'start_year', 'runtime_minutes'], axis = 1)
movie_basics.head(2)

,primary_title,genres
0,Sunghursh,"Action,Crime,Drama"
1,One Day Before the Rainy Season,"Biography,Drama"


In [13]:
# Step 5
new_bmg = bmg_fill_na
new_bmg = new_bmg.drop(['studio', 'year'], axis = 1)
new_bmg

,title,domestic_gross,foreign_gross,total_gross
0,Toy Story 3,415000000.00,652000000.00,1067000000.00
1,Alice in Wonderland (2010),334200000.00,691300000.00,1025500000.00
2,Harry Potter and the Deathly Hallows Part 1,296000000.00,664300000.00,960300000.00
3,Inception,292600000.00,535700000.00,828300000.00
4,Shrek Forever After,238700000.00,513900000.00,752600000.00
...,...,...,...,...
3382,The Quake,6200.00,NaN,NaN
3383,Edward II (2018 re-release),4800.00,NaN,NaN
3384,El Pacto,2500.00,NaN,NaN
3385,The Swan,2400.00,NaN,NaN


## tn_mb:
   
### Step 1: Convert 'production_budget', 'domestic_gross', and 'worldwide_gross' to int type
### Step 2: Create a column 'foreign_gross' by subtracting 'domestic_gross' from 'worldwide_gross'
### Step 3: Create a new column 'release_month'
### Step 4: Create a 'total_ROI' column
### Step 5: Remove records that have 0.0 for 'domestic_gross' and 'worldwide_gross'
- Out of the 5782 records in the original **tn_mb** DataFrame 5415 remain in the new **tn_mb_no_zeros** DataFrame

In [13]:
print(tn_mb.info())
tn_mb.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB
None


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"


In [14]:
# Step 1
tn_mb['production_budget'] = tn_mb['production_budget'].replace('[\$,]', '', regex=True).astype(float)
tn_mb['domestic_gross'] = tn_mb['domestic_gross'].replace('[\$,]', '', regex=True).astype(float)
tn_mb['worldwide_gross'] = tn_mb['worldwide_gross'].replace('[\$,]', '', regex=True).astype(float)

# Step 2
tn_mb['foreign_gross'] = tn_mb['worldwide_gross'] - tn_mb['domestic_gross']

# Step 3
tn_mb['release_month'] = tn_mb['release_date'].str[:3]

#Step 4
tn_mb['total_ROI'] = tn_mb['worldwide_gross'] - tn_mb['production_budget']

# Check to see if there are any missing records
tn_mb.sort_values(by='domestic_gross', ascending = True).head(20)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,release_month,total_ROI
3231,32,"Dec 31, 2008",Ca$h,13700000.00,0.00,11738256.00,11738256.00,Dec,-1961744.00
4762,63,"Dec 31, 2009",Chicago Overcoat,3000000.00,0.00,0.00,0.00,Dec,-3000000.00
4763,64,"Sep 13, 2011",Never Back Down 2: The Beatdown,3000000.00,0.00,0.00,0.00,Sep,-3000000.00
4764,65,"Dec 31, 2015",AWOL-72,3000000.00,0.00,0.00,0.00,Dec,-3000000.00
4194,95,"May 10, 2011",The Hit List,6000000.00,0.00,0.00,0.00,May,-6000000.00
4765,66,"Jan 15, 2013",30 Nights Of Paranormal Activity With The Devi...,3000000.00,0.00,0.00,0.00,Jan,-3000000.00
4766,67,"Feb 6, 2015",Enter the Dangerous Mind,3000000.00,0.00,0.00,0.00,Feb,-3000000.00
5207,8,"Dec 31, 2013",The Vatican Exorcisms,1100000.00,0.00,0.00,0.00,Dec,-1100000.00
5206,7,"Dec 31, 2010",The Dead Undead,1100000.00,0.00,0.00,0.00,Dec,-1100000.00
5205,6,"Apr 7, 2015",Silent Running,1100000.00,0.00,0.00,0.00,Apr,-1100000.00


In [15]:
tn_mb_no_zeros = tn_mb
tn_mb_no_zeros = tn_mb_no_zeros[tn_mb_no_zeros['domestic_gross' and 'worldwide_gross'] != 0.0]
print(tn_mb_no_zeros.info())
tn_mb_no_zeros

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5415 entries, 0 to 5781
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5415 non-null   int64  
 1   release_date       5415 non-null   object 
 2   movie              5415 non-null   object 
 3   production_budget  5415 non-null   float64
 4   domestic_gross     5415 non-null   float64
 5   worldwide_gross    5415 non-null   float64
 6   foreign_gross      5415 non-null   float64
 7   release_month      5415 non-null   object 
 8   total_ROI          5415 non-null   float64
dtypes: float64(5), int64(1), object(3)
memory usage: 423.0+ KB
None


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,release_month,total_ROI
0,1,"Dec 18, 2009",Avatar,425000000.00,760507625.00,2776345279.00,2015837654.00,Dec,2351345279.00
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000.00,241063875.00,1045663875.00,804600000.00,May,635063875.00
2,3,"Jun 7, 2019",Dark Phoenix,350000000.00,42762350.00,149762350.00,107000000.00,Jun,-200237650.00
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000.00,459005868.00,1403013963.00,944008095.00,May,1072413963.00
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000.00,620181382.00,1316721747.00,696540365.00,Dec,999721747.00
...,...,...,...,...,...,...,...,...,...
5775,76,"May 26, 2006",Cavite,7000.00,70071.00,71644.00,1573.00,May,64644.00
5776,77,"Dec 31, 2004",The Mongol King,7000.00,900.00,900.00,0.00,Dec,-6100.00
5778,79,"Apr 2, 1999",Following,6000.00,48482.00,240495.00,192013.00,Apr,234495.00
5779,80,"Jul 13, 2005",Return to the Land of Wonders,5000.00,1338.00,1338.00,0.00,Jul,-3662.00


In [16]:
new_tn_mb = tn_mb_no_zeros
new_tn_mb = new_tn_mb.drop(columns=['id','release_date'])
new_tn_mb

,movie,production_budget,domestic_gross,worldwide_gross,foreign_gross,release_month,total_ROI
0,Avatar,425000000.00,760507625.00,2776345279.00,2015837654.00,Dec,2351345279.00
1,Pirates of the Caribbean: On Stranger Tides,410600000.00,241063875.00,1045663875.00,804600000.00,May,635063875.00
2,Dark Phoenix,350000000.00,42762350.00,149762350.00,107000000.00,Jun,-200237650.00
3,Avengers: Age of Ultron,330600000.00,459005868.00,1403013963.00,944008095.00,May,1072413963.00
4,Star Wars Ep. VIII: The Last Jedi,317000000.00,620181382.00,1316721747.00,696540365.00,Dec,999721747.00
...,...,...,...,...,...,...,...
5775,Cavite,7000.00,70071.00,71644.00,1573.00,May,64644.00
5776,The Mongol King,7000.00,900.00,900.00,0.00,Dec,-6100.00
5778,Following,6000.00,48482.00,240495.00,192013.00,Apr,234495.00
5779,Return to the Land of Wonders,5000.00,1338.00,1338.00,0.00,Jul,-3662.00


In [19]:
close(conn)

NameError: name 'close' is not defined

In [ ]:
a